In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
dataset=pd.read_csv('/content/wholesalepriceindex.csv')


In [2]:
print(dataset.shape)

(869, 142)


In [3]:


wpi_columns = [col for col in dataset.columns if col.startswith('INDX')]

dataset_melted = dataset.melt(id_vars=['COMM_NAME', 'COMM_CODE', 'COMM_WT'],
                               value_vars=wpi_columns,
                               var_name='Date',
                               value_name='WPI_Value')

dataset_melted['Date'] = pd.to_datetime(dataset_melted['Date'].str[4:], format='%m%Y')

dataset_melted = dataset_melted.sort_values(by=['COMM_NAME', 'Date'])



In [4]:
print(dataset_melted)


                  COMM_NAME   COMM_CODE   COMM_WT       Date  WPI_Value
181     (A).  FOOD ARTICLES  1101000000  15.25585 2012-04-01      106.2
1050    (A).  FOOD ARTICLES  1101000000  15.25585 2012-05-01      106.7
1919    (A).  FOOD ARTICLES  1101000000  15.25585 2012-06-01      109.3
2788    (A).  FOOD ARTICLES  1101000000  15.25585 2012-07-01      111.3
3657    (A).  FOOD ARTICLES  1101000000  15.25585 2012-08-01      111.2
...                     ...         ...       ...        ...        ...
116733          slag cement  1313050004   0.09830 2023-06-01      142.4
117602          slag cement  1313050004   0.09830 2023-07-01      142.7
118471          slag cement  1313050004   0.09830 2023-08-01      141.4
119340          slag cement  1313050004   0.09830 2023-09-01      151.6
120209          slag cement  1313050004   0.09830 2023-10-01      151.1

[120791 rows x 5 columns]


In [5]:
dataset_melted['Date'] = pd.to_datetime(dataset_melted['Date'])
dataset_melted['month']=dataset_melted['Date'].dt.month
dataset_melted['year']=dataset_melted['Date'].dt.year
print(dataset_melted)

                  COMM_NAME   COMM_CODE   COMM_WT       Date  WPI_Value  \
181     (A).  FOOD ARTICLES  1101000000  15.25585 2012-04-01      106.2   
1050    (A).  FOOD ARTICLES  1101000000  15.25585 2012-05-01      106.7   
1919    (A).  FOOD ARTICLES  1101000000  15.25585 2012-06-01      109.3   
2788    (A).  FOOD ARTICLES  1101000000  15.25585 2012-07-01      111.3   
3657    (A).  FOOD ARTICLES  1101000000  15.25585 2012-08-01      111.2   
...                     ...         ...       ...        ...        ...   
116733          slag cement  1313050004   0.09830 2023-06-01      142.4   
117602          slag cement  1313050004   0.09830 2023-07-01      142.7   
118471          slag cement  1313050004   0.09830 2023-08-01      141.4   
119340          slag cement  1313050004   0.09830 2023-09-01      151.6   
120209          slag cement  1313050004   0.09830 2023-10-01      151.1   

        month  year  
181         4  2012  
1050        5  2012  
1919        6  2012  
2788       

In [6]:
X=dataset_melted.drop(columns=['COMM_NAME','WPI_Value','Date'])
y=dataset_melted.iloc[:,-3].values
X=sm.add_constant(X)

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

X_train.head()

,const,COMM_CODE,COMM_WT,month,year
87814,1.0,1301060000,2.00952,9,2020
36347,1.0,1301040004,0.23133,9,2015
97170,1.0,1202000004,0.31991,7,2021
15590,1.0,1317010013,0.08079,9,2013
44357,1.0,1202000002,1.60470,7,2016


In [15]:
X_train=np.array(X_train)
X_test=np.array(X_test)
from sklearn.impute import SimpleImputer
impute=SimpleImputer(missing_values=np.nan,strategy='mean')
impute.fit(X_train[:,2:3])
X_train[:,2:3]=impute.transform(X_train[:,2:3])
impute.fit(X_test[:,2:3])
X_test[:,2:3]=impute.transform(X_test[:,2:3])


In [16]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
X_train[:,1:2]=st.fit_transform(X_train[:,1:2])
X_test[:,1:2]=st.transform(X_test[:,1:2])
X_train[:,4:5]=st.fit_transform(X_train[:,4:5])
X_test[:,4:5]=st.transform(X_test[:,4:5])

In [17]:
model=sm.OLS(y_train,X_train).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     6307.
Date:                Sat, 31 May 2025   Prob (F-statistic):               0.00
Time:                        15:12:43   Log-Likelihood:            -4.6220e+05
No. Observations:               96632   AIC:                         9.244e+05
Df Residuals:                   96627   BIC:                         9.245e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        121.3832      0.202    599.684      0.0

In [18]:
columns=['const','x1','x2','x3','x4']
X_train=pd.DataFrame(X_train,columns=columns)
while True:
  max_pvalue=model.pvalues.max()
  if max_pvalue>0.05:
    remove_variable=model.pvalues.idxmax()
    X_train=X_train.drop(columns=[remove_variable])
    model=sm.OLS(y_train,X_train).fit()
  else:
    break



In [19]:
y_predic=model.predict(X_test)

print(y_predic.shape)


(24159,)


In [20]:
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_predic})
comparison.head(40)

,Actual,Predicted
0,126.9,108.110272
1,99.3,111.378085
2,148.4,134.386660
3,158.9,130.012955
4,105.7,104.175805
5,138.3,139.183678
6,118.4,119.936959
7,134.2,137.164396
8,91.1,128.504462
9,126.8,105.162002
